In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import sys

import dash
from dash.dependencies import Input, Output, State
from dash import dcc, html
from openai import OpenAI
import os
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt


print(os.getcwd())

c:\Users\junio\OneDrive\바탕 화면\dash_v2\may


In [3]:
data = pd.read_csv("Customer_info.csv")

In [5]:
data2 = pd.read_csv("Onlinesales_info.csv")

In [7]:
data


,고객ID,성별,고객지역,가입기간
0,USER_1358,남,Chicago,12
1,USER_0190,남,California,43
2,USER_0066,남,Chicago,33
3,USER_0345,여,California,30
4,USER_0683,남,California,49
...,...,...,...,...
1463,USER_0513,여,New York,41
1464,USER_0167,여,Chicago,48
1465,USER_0845,남,New Jersey,19
1466,USER_0504,여,New York,45


In [12]:
data2

,고객ID,거래ID,거래날짜,제품ID,제품카테고리,수량,평균금액,배송료,쿠폰상태
0,USER_1358,Transaction_0000,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used
1,USER_1358,Transaction_0001,2019-01-01,Product_0981,Nest-USA,1,153.71,6.50,Used
2,USER_1358,Transaction_0002,2019-01-01,Product_0904,Office,1,2.05,6.50,Used
3,USER_1358,Transaction_0003,2019-01-01,Product_0203,Apparel,5,17.53,6.50,Not Used
4,USER_1358,Transaction_0003,2019-01-01,Product_0848,Bags,1,16.50,6.50,Used
...,...,...,...,...,...,...,...,...,...
52919,USER_0504,Transaction_25056,2019-12-31,Product_0976,Nest-USA,1,121.30,6.50,Clicked
52920,USER_0504,Transaction_25057,2019-12-31,Product_0413,Apparel,1,48.92,6.50,Used
52921,USER_0504,Transaction_25058,2019-12-31,Product_0989,Nest-USA,1,151.88,6.50,Used
52922,USER_0562,Transaction_25059,2019-12-31,Product_0985,Nest-USA,5,80.52,6.50,Clicked


In [15]:
round(len(data2)*0.3)

15877

In [11]:
import dash
from dash import dcc, html, Input, Output, State
import openai
import dash_bootstrap_components as dbc

# OpenAI API 키 설정
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Dash 앱 초기화
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

# 로그인 레이아웃
login_layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.Container([
        dbc.Row(dbc.Col(html.H2("Login", className="text-center mt-4"))),
        dbc.Row(dbc.Col(dbc.Input(id='username', placeholder='Enter your username', type='text', className="mb-3"))),
        dbc.Row(dbc.Col(dbc.Input(id='password', placeholder='Enter your password', type='password', className="mb-3"))),
        dbc.Row(dbc.Col(dbc.Button('Login', id='login-button', color='primary', className="mb-3"))),
        dbc.Row(dbc.Col(html.Div(id='login-output', className="text-danger")))
    ])
])

# 대시보드 레이아웃 (기본 페이지)
dashboard_layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dbc.Container([
        dbc.Row(dbc.Col(html.H2("Dashboard", className="text-center mt-4"))),
        dbc.Row(dbc.Col(dcc.Input(id='user-input', placeholder='Enter your prompt here', type='text', className="mb-3"))),
        dbc.Row(dbc.Col(dbc.Button('Submit', id='submit-button', color='primary', className="mb-3"))),
        dbc.Row(dbc.Col(html.Div(id='category-output', className="mb-3"))),
        dbc.Row(dbc.Col(html.Div(id='dashboard-content')))
    ])
])

# 전체 앱 레이아웃
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# 페이지 로드 시 레이아웃 결정
@app.callback(Output('page-content', 'children'),
              Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/' or pathname == '/login':
        return login_layout
    elif pathname == '/dashboard':
        return dashboard_layout
    else:
        return '404'

# 로그인 버튼 콜백
@app.callback(Output('url', 'pathname'),
              Output('login-output', 'children'),
              Input('login-button', 'n_clicks'),
              State('username', 'value'),
              State('password', 'value'))
def login(n_clicks, username, password):
    if n_clicks:
        if username == 'admin' and password == 'password':  # 간단한 인증 로직
            return '/dashboard', ''
        else:
            return '/login', 'Invalid username or password'
    return '/login', ''

# 프롬프트 제출 버튼 콜백
@app.callback(Output('category-output', 'children'),
              Output('dashboard-content', 'children'),
              Input('submit-button', 'n_clicks'),
              State('user-input', 'value'))
def classify_prompt(n_clicks, user_input):
    if n_clicks and user_input:
        response = openai.Completion.create(
            model="text-davinci-003",
            prompt=f"다음 내용을 카테고리로 분류하세요: {user_input}\n\n카테고리: [이탈률, 매출액, 마케팅, 재구매, ROI, 해당없음]",
            max_tokens=10
        )
        category = response.choices[0].text.strip()

        if category == '이탈률':
            return category, html.Div([
                html.H3("이탈률 대시보드"),
                html.P("이탈률 관련 데이터를 보여줍니다."),
                dcc.Graph(
                    figure={
                        'data': [{'x': [1, 2, 3], 'y': [10, 20, 30], 'type': 'line', 'name': '이탈률'}],
                        'layout': {'title': '이탈률 그래프'}
                    }
                )
            ])
        elif category == '매출액':
            return category, html.Div([
                html.H3("매출액 대시보드"),
                html.P("매출액 관련 데이터를 보여줍니다."),
                dcc.Graph(
                    figure={
                        'data': [{'x': [1, 2, 3], 'y': [100, 200, 300], 'type': 'bar', 'name': '매출액'}],
                        'layout': {'title': '매출액 그래프'}
                    }
                )
            ])
        elif category == '마케팅':
            return category, html.Div([
                html.H3("마케팅 대시보드"),
                html.P("마케팅 관련 데이터를 보여줍니다."),
                dcc.Graph(
                    figure={
                        'data': [{'x': ['캠페인 A', '캠페인 B', '캠페인 C'], 'y': [1000, 1500, 2000], 'type': 'bar', 'name': '마케팅'}],
                        'layout': {'title': '마케팅 캠페인 성과'}
                    }
                )
            ])
        elif category == '재구매':
            return category, html.Div([
                html.H3("재구매 대시보드"),
                html.P("재구매 관련 데이터를 보여줍니다."),
                dcc.Graph(
                    figure={
                        'data': [{'x': [1, 2, 3], 'y': [50, 70, 90], 'type': 'line', 'name': '재구매율'}],
                        'layout': {'title': '재구매율 그래프'}
                    }
                )
            ])
        elif category == 'ROI':
            return category, html.Div([
                html.H3("ROI 대시보드"),
                html.P("ROI 관련 데이터를 보여줍니다."),
                dcc.Graph(
                    figure={
                        'data': [{'x': ['캠페인 A', '캠페인 B', '캠페인 C'], 'y': [1.2, 1.5, 1.8], 'type': 'bar', 'name': 'ROI'}],
                        'layout': {'title': 'ROI 그래프'}
                    }
                )
            ])
        else:
            return category, html.Div([
                html.H3("해당없음"),
                html.P("해당 카테고리에 대한 데이터가 없습니다.")
            ])
    return '', ''

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True)